In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/17 21:13:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-11 04:05:54|  2020-01-11 04:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.

In [4]:
from pyspark.sql import functions as F


In [9]:
# Perform the same transformations and calculations using DataFrame API
df_green_revenue = df_green \
    .withColumn('hour', F.date_trunc('hour', F.col('lpep_pickup_datetime'))) \
    .filter(F.col('lpep_pickup_datetime') >= '2020-01-01 00:00:00') \
    .groupBy('hour', 'PULocationID') \
    .agg(
        F.sum('total_amount').alias('amount'),
        F.count('*').alias('number_records')  # Count all rows in the group
    )


In [10]:
df_green_revenue.show()

+-------------------+------------+------------------+--------------+
|               hour|PULocationID|            amount|number_records|
+-------------------+------------+------------------+--------------+
|2020-01-22 18:00:00|          49|            163.25|             8|
|2020-01-23 13:00:00|         260|            135.01|             9|
|2020-01-05 19:00:00|         216|            249.89|             7|
|2020-01-16 13:00:00|          82| 309.4800000000002|            30|
|2020-01-17 08:00:00|         116|333.40000000000003|            20|
|2020-01-05 14:00:00|          74|1208.1699999999987|            87|
|2020-01-16 22:00:00|          35|             64.75|             2|
|2020-01-08 20:00:00|          65| 447.3200000000001|            32|
|2020-01-12 04:00:00|          80|            178.57|            11|
|2020-01-22 09:00:00|          74|1390.3599999999988|           101|
|2020-01-04 08:00:00|         260|38.150000000000006|             4|
|2020-01-22 14:00:00|          25|

In [11]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [16]:
# Perform the same transformations and calculations using DataFrame API
df_yellow_revenue = df_yellow \
    .withColumn('hour', F.date_trunc('hour', F.col('tpep_pickup_datetime'))) \
    .filter(F.col('tpep_pickup_datetime') >= '2020-01-01 00:00:00') \
    .groupBy('hour', 'PULocationID') \
    .agg(
        F.sum('total_amount').alias('amount'),
        F.count('*').alias('number_records')  # Count all rows in the group
    )

In [17]:
df_yellow_revenue.show()

+-------------------+------------+------------------+--------------+
|               hour|PULocationID|            amount|number_records|
+-------------------+------------+------------------+--------------+
|2020-01-14 14:00:00|         138|14898.500000000011|           339|
|2020-01-11 22:00:00|         132|18350.820000000014|           355|
|2020-01-22 16:00:00|         230| 9063.570000000022|           507|
|2020-01-12 02:00:00|           4|1224.3199999999997|            71|
|2020-01-13 11:00:00|         237| 7470.460000000018|           550|
|2020-01-04 17:00:00|         143|2078.5999999999995|           135|
|2020-01-28 19:00:00|         238|3897.2200000000016|           247|
|2020-01-08 08:00:00|          68|           3530.24|           223|
|2020-01-20 13:00:00|         163| 6030.090000000009|           345|
|2020-01-21 08:00:00|         113|3592.9900000000002|           225|
|2020-01-20 19:00:00|          48|4079.9000000000024|           262|
|2020-01-26 12:00:00|         113|

In [18]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [19]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [22]:
df_green_revenue.columns

['hour', 'PULocationID', 'amount', 'number_records']

In [23]:
df_yellow_revenue.columns

['hour', 'PULocationID', 'amount', 'number_records']

In [24]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [25]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'PULocationID'], how='outer')

In [26]:
df_join.show()

+-------------------+------------+------------------+--------------------+------------------+---------------------+
|               hour|PULocationID|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+------------+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|          10|              null|                null|             42.41|                    2|
|2020-01-01 00:00:00|          17|            195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|          35|            129.96|                   5|              null|                 null|
|2020-01-01 00:00:00|          36|295.34000000000003|                  11|            109.17|                    3|
|2020-01-01 00:00:00|          42| 799.7599999999992|                  52| 635.3500000000004|                   46|
|2020-01-01 00:00:00|          45|              null|                nul

In [27]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [28]:
df_join = spark.read.parquet('data/report/revenue/total')

In [29]:
df_join.show()

+-------------------+------------+------------------+--------------------+------------------+---------------------+
|               hour|PULocationID|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+------------+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|           4|              null|                null|            1004.3|                   57|
|2020-01-01 00:00:00|           7| 769.7299999999996|                  45| 455.1700000000002|                   38|
|2020-01-01 00:00:00|          22|              15.8|                   1|              null|                 null|
|2020-01-01 00:00:00|          25|             531.0|                  26|324.34999999999997|                   16|
|2020-01-01 00:00:00|          33|317.27000000000004|                  11|            255.56|                    8|
|2020-01-01 00:00:00|          34|              null|                nul

In [31]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2024-12-17 23:38:51--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.72, 54.230.209.140, 54.230.209.126, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2024-12-17 23:38:51 (19.7 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [33]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [34]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [35]:
df_zones.write.parquet('zones')

In [36]:
df_zones = spark.read.parquet('zones/')

In [37]:
df_join.columns

['hour',
 'PULocationID',
 'green_amount',
 'green_number_records',
 'yellow_amount',
 'yellow_number_records']

In [38]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [39]:
df_result = df_join.join(df_zones, df_join.PULocationID == df_zones.LocationID)

In [40]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [42]:
df_result.columns

['hour',
 'PULocationID',
 'green_amount',
 'green_number_records',
 'yellow_amount',
 'yellow_number_records',
 'LocationID',
 'Borough',
 'Zone',
 'service_zone']

In [43]:
spark.stop()